# End-Word
Combine an Excel data table with Word text and images
- Literally just copy and pasting from one doc to another, but in a very convoluted way using the OML data structure underlying Office


In [1]:
'''For jupyter notebook usage only - enables cells to be collapsed in Jupyter notebook
'''
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

print('hide_toggle() definition')
hide_toggle()

hide_toggle() definition


In [2]:
# Standard imports
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os, shutil, io, datetime
from pprint import pprint

# Docx imports - for Word
import docx  # To read docx and extract data
from docxtpl import DocxTemplate, InlineImage  # To pass images to new doc
from docx.shared import Mm, Inches, Pt  # To preserve image sizes
from docx.enum.text import WD_ALIGN_PARAGRAPH, WD_BREAK  # To get paragraph justification types
from docx.enum.section import WD_SECTION  # To get word sections
from docx.enum.dml import MSO_THEME_COLOR # Get theme colors
from docx.oxml import OxmlElement  # For defining and targeting xml elements to change
from docx.oxml.ns import qn  # For defining and targeting xml elements to change
from docx.text.paragraph import Paragraph
from docxcompose.composer import Composer  # Append files together, preserving everything except sections

# Openpyxl imports - for Excel
from openpyxl import load_workbook
from openpyxl import styles

# Local imports

print('Imports')
hide_toggle()

Imports


In [3]:
# Helper functions
def get_para_data(dest, src_p):
    """
    Write the run to the new file and then set its font, bold, alignment, color etc. data.
    
    More text attributes: https://python-docx.readthedocs.io/en/latest/api/text.html
    """
    dest_para = dest.add_paragraph(style=src_p.style.name)

    for run in src_p.runs:
        
        dest_run = dest_para.add_run(run.text)
        
        # Apply text styles
        dest_run.bold = run.bold
        dest_run.italic = run.italic
        dest_run.underline = run.underline
        dest_run.style.name = run.style.name
        dest_run.font.color.rgb = run.font.color.rgb
        dest_run.font.name = run.font.name
        dest_run.font.subscript = run.font.subscript
        dest_run.font.superscript = run.font.superscript
        dest_run.font.size = run.font.size
        
        # Add run for footnote
        if run.footnote:
            dest_para.add_footnote(run.footnote)
        
    # Align paragraph
    dest_para.alignment = src_p.alignment
        

def new_section_cols(dest, num_cols):
    new_section = dest.add_section(WD_SECTION.CONTINUOUS)
    sectPr = new_section._sectPr
    cols = OxmlElement('w:cols')
    cols.set(qn('w:num'), str(num_cols))
    sectPr.append(cols)
    
print('Package helper functions')
hide_toggle()

Package helper functions


In [38]:
def style_tbl(table, xls_formats):
    # Helpers
    def borders(xls_format):
        tcBorders = OxmlElement('w:tcBorders')
        for position in ['top', 'bottom', 'left', 'right']:
                
            # Map xls border format to xml format
            if xls_format['border'][position] == 'thin':
                val = 'single'
                sz = '2'
            elif xls_format['border'][position] == 'medium':
                val = 'single'
                sz = '8'
            elif xls_format['border'][position] == 'thick':
                val = 'single'
                sz = '16'
            else:
                val = 'nil'
                sz = '2'
            
            # Set border formats on obj
            # More options at http://officeopenxml.com/WPtableBorders.php
            side = OxmlElement(f'w:{position}')
            side.set(qn('w:val'), val)
            side.set(qn('w:sz'), sz)  # sz 2 = 1/4 pt is the minimum
            side.set(qn('w:space'), '0')
            side.set(qn('w:shadow'), 'false')
            if xls_format['border'][f'{position}Color'] is not None:  # Catch Nonetype colors
                side.set(qn('w:color'), xls_format['border'][f'{position}Color'])
                
            tcBorders.append(side)
        tcPr.append(tcBorders)
        

    def fill_align(xls_format):
        # https://docs.microsoft.com/en-us/dotnet/api/documentformat.openxml.wordprocessing.shading?view=openxml-2.8.1
        # Set cell fill 
        fillshade = OxmlElement('w:shd')
        fillshade.set(qn('w:fill'), xls_format['fillColor'])
        tcPr.append(fillshade)
        
        # Set alignment
        vAlign = OxmlElement('w:vAlign')
        try:
            vAlign.set(qn('w:val'), xls_format['vertical'])
            tcPr.append(vAlign)
        except TypeError:
#             print(f'No vertical alignment for a table @ cell {coord} - skipping...')
            pass
        
        
    def fonts(xls_format):
        # https://python-docx.readthedocs.io/en/latest/dev/analysis/features/text/font-color.html
        try:
            run = cell.p_lst[0].r_lst[0]
        except IndexError as e:
            # print(f'{e}: No run in cell {coord} - skipping')
            return
        rPr = run._add_rPr()
        # Set font color
        if xls_format['fontColor']:
            fontColor = OxmlElement('w:color')
            fontColor.set(qn('w:val'), xls_format['fontColor'])
            rPr.append(fontColor)
        # Set bold
        if xls_format['bold']:
            fontBold = OxmlElement('w:b')
            rPr.append(fontBold)
        # Set font size
        if xls_format['size']:
            size_val = xls_format['size'] * 2  # Measurements are in half-points
            size_val = str(int(size_val))
            fontSize = OxmlElement('w:sz')
            fontSize_cs = OxmlElement('w:szCs')
            fontSize.set(qn('w:val'), size_val)
            fontSize_cs.set(qn('w:val'), size_val)
            rPr.append(fontSize)
            rPr.append(fontSize_cs)
        # Set font name
        if xls_format['name']:
            fontName = OxmlElement('w:rFonts')
            fontName.set(qn('w:ascii'),xls_format['name'])
            fontName.set(qn('w:hAnsi'),xls_format['name'])
            fontName.set(qn('w:cs'),xls_format['name'])
            rPr.append(fontName)
    
    def check_merge(tcPr, merged_cell_count, merged_cell_flag):
        # Note: merged_cell_flag is set to false for the first cell in a merged group
        # If the cell is the start of a new merge, set flags to true and reset merge counter
        if tcPr.gridSpan is not None:
            merged_cell_count = tcPr.grid_span
        # Else if the cell is merged, reduce merged_cell_count by 1
        elif merged_cell_count > 0:
            merged_cell_count -= 1
            merged_cell_flag = True
        # Else if the cell is no longer merged, set merge cell flag to false
        elif merged_cell_count == 0:
            merged_cell_flag = False
        else:
            raise LogicError('This condition should never be hit')
        return merged_cell_flag, merged_cell_count
    
    
    # Main
    tbl = table._tbl # get xml element of the table
    merged_cell_flag = False  # initiate merged cell flag to mark if cell is part of marged group
    merged_cell_count = 0  # initiate counter for merged cells
    
    for cell in tbl.iter_tcs():
        coord = (cell.bottom-1, cell._grid_col)
        tcPr = cell.tcPr  # Get table cell properties
        merged_cell_flag, merged_cell_count = check_merge(tcPr, merged_cell_count, merged_cell_flag)
        
        # FOR TESTING: If it is part of a merged cell, go to next cell in loop
#         if merged_cell_flag:
#             print(f'Skipped cell {coord}')
#             continue
        
        # Run style changes
        borders(xls_formats[coord])
        fill_align(xls_formats[coord])
        fonts(xls_formats[coord])


print('style_tbl() helper function')
hide_toggle()

style_tbl() helper function


In [5]:
# Convert openpyxl theme colours to rgb...
from colorsys import rgb_to_hls, hls_to_rgb

RGBMAX = 0xff  # Corresponds to 255
HLSMAX = 240  # MS excel's tint function expects that HLS is base 240. see:
# https://social.msdn.microsoft.com/Forums/en-US/e9d8c136-6d62-4098-9b1b-dac786149f43/excel-color-tint-algorithm-incorrect?forum=os_binaryfile#d3c2ac95-52e0-476b-86f1-e2a697f24969

def rgb_to_ms_hls(red, green=None, blue=None):
    """Converts rgb values in range (0,1) or a hex string of the form '[#aa]rrggbb' to HLSMAX based HLS, (alpha values are ignored)"""
    if green is None:
        if isinstance(red, str):
            if red == '00000000':
                return (HLSMAX,HLSMAX,HLSMAX)  # Alpha is 0, so cell is transparent.
            if len(red) > 6:
                red = red[-6:]  # Ignore preceding '#' and alpha values
            blue = int(red[4:], 16) / RGBMAX
            green = int(red[2:4], 16) / RGBMAX
            red = int(red[0:2], 16) / RGBMAX
        else:
            red, green, blue = red
    h, l, s = rgb_to_hls(red, green, blue)
    return (int(round(h * HLSMAX)), int(round(l * HLSMAX)), int(round(s * HLSMAX)))

def ms_hls_to_rgb(hue, lightness=None, saturation=None):
    """Converts HLSMAX based HLS values to rgb values in the range (0,1)"""
    if lightness is None:
        hue, lightness, saturation = hue
        result = hls_to_rgb(hue / HLSMAX, lightness / HLSMAX, saturation / HLSMAX)
        print(result)
    return hls_to_rgb(hue / HLSMAX, lightness / HLSMAX, saturation / HLSMAX)

def rgb_to_hex(red, green=None, blue=None):
    """Converts (0,1) based RGB values to a hex string 'rrggbb'"""
    if green is None:
        red, green, blue = red
    return ('%02x%02x%02x' % (int(round(red * RGBMAX)), int(round(green * RGBMAX)), int(round(blue * RGBMAX)))).upper()


def get_theme_colors(wb):
    """Gets theme colors from the workbook"""
    # see: https://groups.google.com/forum/#!topic/openpyxl-users/I0k3TfqNLrc
    from openpyxl.xml.functions import QName, fromstring
    xlmns = 'http://schemas.openxmlformats.org/drawingml/2006/main'
    root = fromstring(wb.loaded_theme)
    themeEl = root.find(QName(xlmns, 'themeElements').text)
    colorSchemes = themeEl.findall(QName(xlmns, 'clrScheme').text)
    firstColorScheme = colorSchemes[0]

    colors = []

    for c in ['lt1', 'dk1', 'lt2', 'dk2', 'accent1', 'accent2', 'accent3', 'accent4', 'accent5', 'accent6']:
        accent = firstColorScheme.find(QName(xlmns, c).text)

        if 'window' in accent.getchildren()[0].attrib['val']:
            colors.append(accent.getchildren()[0].attrib['lastClr'])
        else:
            colors.append(accent.getchildren()[0].attrib['val'])

    return colors

def tint_luminance(tint, lum):
    """Tints a HLSMAX based luminance"""
    # See: http://ciintelligence.blogspot.co.uk/2012/02/converting-excel-theme-color-and-tint.html
    if tint < 0:
        return int(round(lum * (1.0 + tint)))
    else:
        return int(round(lum * (1.0 - tint) + (HLSMAX - HLSMAX * (1.0 - tint))))

def theme_and_tint_to_rgb(wb, theme, tint):
    """Given a workbook, a theme number and a tint return a hex based rgb"""
    rgb = get_theme_colors(wb)[theme]
    h, l, s = rgb_to_ms_hls(rgb)
    return rgb_to_hex(ms_hls_to_rgb(h, tint_luminance(tint, l), s))

def ms_rgb_to_hex_rgb(ms_rgb, tint):
    '''Give a hex string of the form '[#aa]rrggbb' and tint, return a hex based rgb'''
    h, l, s = rgb_to_ms_hls(ms_rgb)
    return rgb_to_hex(ms_hls_to_rgb(h, tint_luminance(tint, l), s))

print('Colour conversion code for Theme-Tint to RGB')
hide_toggle()

Colour conversion code for Theme-Tint to RGB


In [6]:
# Written by Reddit user _DTR_

import re
import xml.etree.ElementTree as ET
import zipfile


# The following prefixes are prepended to xml tags within xlsx files.
# Make our lives easier and give them their own variables
PREFIX = '{http://schemas.openxmlformats.org/spreadsheetml/2006/main}'
REL = '{http://schemas.openxmlformats.org/officeDocument/2006/relationships}'

'''
Class that contains the overarching workbook. Very simple at the moment, containing only a
dictionary of sheets and the currently active one
'''
class Workbook:
    def __init__(self):
        self.sheets = {}
        self.active = None

    def add_sheet(self, sheet, name, active=False):
        self.sheets[name] = sheet
        if active:
            self.active = sheet

'''
The worksheet class that holds the cell values
'''
class Worksheet:
    def __init__(self):
        self.cells = {}
        self.def_cell = SharedString()

        # Keep things 1-based
        self.dim = { 'rw_first' : 1, 'rw_last' : 1, 'col_first' : 1, 'col_last' : 1 }

    '''
    Adds a cell at the given A1 reference
    '''
    def add_cell(self, location, string):
        # self.cells[location] = string
        rw, col = CellHelpers.rwcol_from_ref(location)
        if not rw in self.cells:
            self.cells[rw] = {}

        self.cells[rw][col] = string

        self.dim['rw_first'] = min(rw, self.dim['rw_first'])
        self.dim['rw_last'] = max(rw, self.dim['rw_last'])
        self.dim['col_first'] = min(col, self.dim['col_first'])
        self.dim['col_last'] = max(col, self.dim['col_last'])

    '''
    Retrieve a cell at the given (1-based) row and column
    '''
    def cell(self, rw, col):
        return CellHolder(rw, col, self._cell(rw, col))

    '''
    Retrieve a range of cells determined by the A1 reference string rng
    By default, cells are gathered left-to-right, top-to-bottom (row-major order)
    To iterate by columns first (top-down, left-to-right), set row_major to false
    Handled strings:
        - Single cell ('A4', 'XFD10000') - returns a list of length one, containing the given cell
        - General range ('A1:B2') - returns a two-dimensional list of all the cells in the given range
            - If the range is within a single row or single column, return a one dimensional list,
              otherwise a 2 dimensional list (apaplies to the below as well)
        - Entire columns ('A:A', 'C:Z') - returns an array of all cells in the specified rows, starting
          at row 1 to the row of the last non-blank cell in the worksheet
        - Entire rows ('1:1', '3:5') - same as columns, swapped
    '''
    def get_range(self, rng, row_major=True):
        rng = rng.upper()
        sep_index = rng.find(':')
        if sep_index == -1:
            rw, col = _rw_col(rng)
            return [CellHolder(rw, col, self._cell(rw, col))]

        rw_first, rw_last, col_first, col_last = 4 * [1]

        # Need to check for entire rows/cols. These checks aren't foolproof, they aren't really meant to be
        # Assuming valid input, they do the right thing
        if not re.match(r'\w+\d', rng[:sep_index]):
            if (re.match(r'\w+\d', rng[sep_index + 1:])):
                # invalid, can't have something like A:A1 or A:B5
                return []
            rw_first = 1
            rw_last = self.dim['rw_last']
            col_first = CellHelpers.col_to_num(rng[:sep_index])
            col_last = CellHelpers.col_to_num(rng[sep_index + 1:])
        elif re.match(r'\d+:\d+', rng):
            # Entire column
            rw_first = int(rng[:sep_index])
            rw_last = rw_int(rng[sep_index + 1:])
            col_first = 1
            col_last = self.dim['col_last']
        else:
            rw_first, col_first = CellHelpers.rwcol_from_ref(rng[:sep_index])
            rw_last, col_last = CellHelpers.rwcol_from_ref(rng[sep_index + 1:])

        # Invalid range, return and empty list
        if col_first > col_last or rw_first > rw_last:
            return []

        res = []
        if rw_first == rw_last or col_first == col_last:
            # In the single row/column case, don't return a nested list, just a single one
            for rw in range(rw_first, rw_last + 1):
                for col in range(col_first, col_last + 1):
                    res.append(CellHolder(rw, col, self._cell(rw, col)))
        elif row_major:
            for rw in range(rw_first, rw_last + 1):
                rw_vals = []
                for col in range(col_first, col_last + 1):
                    rw_vals.append(CellHolder(rw, col, self._cell(rw, col)))

                res.append(rw_vals)
        else:
            for col in range(col_first, col_last + 1):
                col_vals = []
                for rw in range(rw_first, rw_last + 1):
                    col_vals.append(CellHolder(rw, col, self._cell(rw, col)))
                res.append(col_vals)

        return res

    '''
    Internal method to return the value at a given cell (or a default blank cell if there is no value)
    '''
    def _cell(self, rw, col):
        if rw in self.cells and col in self.cells[rw]:
            return self.cells[rw][col]
        return self.def_cell

'''
Class containing helper methods for converting between row/col and A1 reference styles
'''
class CellHelpers:

    '''
    Convert the given A1 column to a 1-based index
    '''
    def col_to_num(col):
        if len(col) == 1:
            return ord(col) - ord('A') + 1
        if len(col) == 2:
            return 26 + (26 * (ord(col[0]) - ord('A'))) + (ord(col[1]) - ord('A')) + 1
        else:
            return 702 + (676 * (ord(col[0]) - ord('A'))) + (26 * (ord(col[1]) - ord('A'))) + (ord(col[2]) - ord('A')) + 1

    '''
    Convert a single-cell A1 reference to a row and column
    '''
    def rwcol_from_ref(ref):
        index = 0
        while ord(ref[index]) < ord('0') or ord(ref[index]) > ord('9'):
            index += 1
        return int(ref[index:]), CellHelpers.col_to_num(ref[:index])

    '''
    Convert a column index into an A1 reference
    '''
    def num_to_col(num):
        num -= 1 # We expect a 1-based col, make it 0-based before doing math
        val = ""
        if num >= 702:
            val = chr(((num - 702) // 676) + ord('A'))
            num %= 676

        if num >= 26:
            val += chr(((num - 26) // 26) + ord('A'))
            num %= 26

        return val + chr(num + ord('A'))

    '''
    Convert a row and column index to an A1 reference string
    '''
    def a1(rw, col):
        return CellHelpers.num_to_col(col) + str(rw)

    '''
    Builds a reference string within the four conrer bounds
    '''
    def build_range(rw_first, rw_last, col_first, col_last):
        return CellHelpers.num_to_col(col_first) + str(rw_first) + ":" + CellHelpers.num_to_col(col_last) + str(rw_last)


'''
Simple wrapper for a cell value
'''
class CellHolder:
    def __init__(self, rw, col, value):
        self.value = value
        self.rw = rw
        self.col = col

    def __repr__(self):
        return self.value.plain_text()

'''
SharedString contains the contents of a single cell. The string may contain
multiple runs each with different formatting
'''
class SharedString:
    def __init__(self, text=None, properties=None):
        self.runs = []
        if text != None:
            self.runs.append(Run(plain=text, properties=properties))

    def add_run(self, xmlNode):
        self.runs.append(Run(xmlNode=xmlNode))

    '''
    Return a plain-text representation of the cell value
    '''
    def plain_text(self):
        return ''.join([run.to_string() for run in self.runs])

    '''
    Replace text within the string. Only supports non-spanning find/replace, i.e. if the find string
    spans changes in formatting, it is not replaced
    '''
    def replace(self, find, rep):
        for run in self.runs:
            run.text = run.text.replace(find, rep)

    '''
    Add this string to the given docx paragraph. Probably shouldn't be a part of this class
    '''
    def add_to_paragraph(self, p):
        for run in self.runs:
            docRun = p.add_run(run.to_string())

            # For now, only look at super/subscript, bold, underline, and italic
            if run.has_attr('vertAlign'):
                if run.attrib('vertAlign') == 'subscript':
                    docRun.font.subscript = True
                elif run.attrib('vertAlign') == 'superscript':
                    docRun.font.superscript = True
            if run.has_attr('b'):
                docRun.font.bold = True
            if run.has_attr('u'):
                docRun.font.underline = True
            if run.has_attr('i'):
                docRun.font.italic = True

    def __str__(self):
        return self.plain_text()

    def __repr__(self):
        return self.plain_text()

'''
A single (un)formatted run of text
'''
class Run:
    '''
    Input is either an xml node or plain text. If it's an xml node, it is parsed
    and formatting properties are applied. If it's plain text, add it directly
    without any properties
    '''
    def __init__(self, xmlNode=None, plain=None, properties=None):
        self.text = ''
        self.properties = {}

        if plain != None:
            self.text = plain
            if properties != None:
                self.properties = properties.copy()
            return

        # xmlNode  better not be none!
        if xmlNode.tag == PREFIX + 't':
            # Easy case, a single string
            self.text = xmlNode.text
            return
        elif xmlNode.tag != PREFIX + 'r':
            print("Unknown tag: " + xmlNode.tag[len(PREFIX):])
            return

        for child in xmlNode:
            if child.tag == PREFIX + 't':
                self.text = child.text
            elif child.tag != PREFIX + 'rPr':
                print("Unknown flag: ", child.tag[len(PREFIX):])
                continue
            for prop in child:
                item = prop.tag[len(PREFIX):]
                if len(prop.attrib) == 0:
                    # Some properties (bold/italic/underline/etc) don't have attributes, but we should still get the tag
                    self.properties[item] = True
                else:
                    for attr in prop.attrib:
                        self.properties[item] = prop.attrib[attr]
                        break;

    def to_string(self):
        return self.text

    def attributes(self):
        return self.properties

    def attrib(self, attribute):
        return self.properties[attribute] if attribute in self.properties else None

    def has_attr(self, attr):
        return attr in self.properties

'''
Reads in the given workbook file and returns a Workbook object containing its sheets and cell values
'''
def custom_load_workbook(source_file):

    # This assumes an xlsx file that has all the required parts
    container = zipfile.ZipFile(source_file)
    stringFile = ET.parse(container.open('xl/sharedStrings.xml'))


    # Build up our list of shared strings
    strings = []
    for child in stringFile.getroot():
        text = SharedString()
        for run in child:
            text.add_run(run)
        strings.append(text)

    # Build up our list of styles
    cell_styles = []
    if 'xl/styles.xml' in container.namelist():
        style = ET.parse(container.open('xl/styles.xml'))
        fonts_xml = style.getroot().find(PREFIX + 'fonts')
        fonts = []
        for font in fonts_xml:
            props = {}
            for prop in font:
                item = prop.tag[len(PREFIX):]
                if len(prop.attrib) == 0:
                    props[item] = True
                else:
                    props[prop.tag[len(PREFIX):]] = prop.attrib[list(prop.attrib.keys())[0]]
            fonts.append(props.copy())

        cellxfs = style.getroot().find(PREFIX + 'cellXfs')
        index = 0
        for xf in cellxfs:
            cell_styles.append({})
            # Only care about font properties for now
            if 'fontId' in xf.attrib:
                fid = int(xf.attrib['fontId'])
                cell_styles[index]['font'] = fonts[fid]
            else:
                cell_styles[index]['font'] = {}
            index += 1


    wb = Workbook()

    # Get the friendly names of the sheets
    sheetNames = {}
    workbook = ET.parse(container.open('xl/workbook.xml'))
    elements = workbook.getroot().findall(PREFIX + 'sheets/' + PREFIX + 'sheet')
    for sheet in elements:
        sheetNames[sheet.attrib['sheetId']] = { 'name' : sheet.attrib['name'] }

    # now go through our worksheets and find matching string entries
    xmlSheets = [file for file in container.namelist() if file[0:16] == 'xl/worksheets/sh']
    for xmlSheet in xmlSheets:
        ws = Worksheet()
        name = sheetNames[xmlSheet[xmlSheet.find('/sheet') + 6:xmlSheet.rfind('.')]]['name']
        active = False
        if xmlSheet.find(PREFIX + '/sheetViews' + PREFIX + '/sheetView[@tabSelected="1"]'):
            active = True
        tree = ET.parse(container.open(xmlSheet))
        data = tree.getroot().find(PREFIX + 'sheetData')
        cells = data.findall(PREFIX + 'row/' + PREFIX + 'c')
        for cell in cells:
            cell_value = None
            # Cell attributes:
            # https://docs.microsoft.com/en-us/dotnet/api/documentformat.openxml.spreadsheet.cell?view=openxml-2.8.1
            if ('t' in cell.attrib and cell.attrib['t'] == 's'):
                # We have a shared string
                ws.add_cell(cell.attrib['r'], strings[int(cell.find(PREFIX + 'v').text)])
            else:
                # otherwise we just have the value.
                properties = None
                if ('s' in cell.attrib):
                    style_id = int(cell.attrib['s'])
                    properties = cell_styles[style_id]['font']
                
                # Format numbers as desired
                try:
                    cell_value = cell.find(PREFIX + 'v').text
                    cell_value = f'{float(cell_value):.2f}'
                except AttributeError as e:
                    # Will catch blank cells, i.e. merged cells
#                     print(f'Cell {cell.attrib["r"]} is blank...skipping\n')
                    pass
                
                if cell_value:
                    ws.add_cell(cell.attrib['r'], SharedString(cell_value, properties=properties))
        wb.add_sheet(ws, name, active)
    
    return wb

print('''Excel XML parser module. 
Will need to separate as separate module and relative import later.
Used instead of openpyxl''')
hide_toggle()

Excel XML parser module. 
Will need to separate as separate module and relative import later.
Used instead of openpyxl


## Main Functions

In [7]:
def docx_composer(base, new_docx, new_page=False):
    '''Appends a new docx file to a base DocxTemplate object, and returns the object for further
    
    Supports text, styles, shapes, in-line images and floating images
    DOES NOT support section breaks and columns
    
    Parameters
    ----------
    base : DocxTemplate object
        Base docx file being worked on
    new_docx : str
        The file location of the target docx source
    
    Returns
    -------
    combined_dest : DocxTemplate object
    
    '''
    # New page if true
    if new_page:
        base.add_paragraph().add_run().add_break(WD_BREAK.PAGE)
    
    composer = Composer(base)
    new_doc = docx.Document(new_docx)
    composer.append(new_doc)
    
    temp_file_path = os.path.join(temp_path,'combined.docx')
    composer.save(temp_file_path)
    combined_base = DocxTemplate(temp_file_path)
    
    return combined_base

print('docx_composer()')
hide_toggle()

docx_composer()


In [8]:
def publish():
    
    # Finalise destination
    dest.paragraphs[-1].add_run().add_break(WD_BREAK.PAGE)
    dest.render(context)
    
    # Finalise backpage
    backpage_doc = DocxTemplate(backpage)
    backpage_doc.render(context)
    
    # Combine documents
    composer = Composer(dest)
    composer.append(backpage_doc)
    
    # Save output and delete temp folder with all contents
    composer.save(output_path)
    print(f'Saved at {output_path}')
    shutil.rmtree(temp_path)
    
print('publish()')
hide_toggle()

publish()


In [9]:
def append_xlsx(dest, source, heading=None):
    '''Appends Excel data source to the destination Word doc as a Table
    
    Does not dynamically search for table contents.
    Table data must start in cell A1.
    Only plots one table per Excel workbook.
    
    Parameters
    ----------
    dest : str
        The file location of the destination word doc
    source: str
        The file location of the target Excel source
    heading: str
        A string that will be printed in the style of Heading 1 above the table in word (default is None)
    '''
    # Sub-functions
    def xl2doc_color(color_meta):
        if color_meta is None:
            return
        tint = color_meta.tint
        if color_meta.type == 'theme':
            theme = color_meta.theme
            fillcolor = theme_and_tint_to_rgb(wb, theme, tint)
        elif color_meta.type == 'rgb':
            ms_rgb = color_meta.rgb
            fillcolor = ms_rgb_to_hex_rgb(ms_rgb, tint)
        elif color_meta.type == 'indexed':
            index = color_meta.indexed
            ms_rgb = styles.colors.COLOR_INDEX[index]
            if 'Foreground' in ms_rgb:
                ms_rgb = '00000000'  # Black
            else:
                ms_rgb = 'FF000000' # White
            fillcolor = ms_rgb_to_hex_rgb(ms_rgb, tint)
            
        else:
            raise TypeError(f'Unrecognised color-type: "{color_meta.type}". Check classes')
        return fillcolor

    
    """Copies cell text runs (for subscript and superscript support)
    Hacked solution...can be made better?
    """
    def cell_text_runs():
        wb = custom_load_workbook(source) # Related to Excel XML parser module
        ws = wb.active
        max_col = ws.dim['col_last']
        max_row = ws.dim['rw_last']
        
        # Store cell contents in an array
        cell_contents = []
        
        ws_range = CellHelpers.build_range(1, max_row,1,max_col)
        for r_idx, row in enumerate(ws.get_range(ws_range)):
            row_contents = []
            for c_idx, cell in enumerate(row):
                # Convert cell reference to SharedString pointer
                cell_ref = ws.cell(cell.rw, cell.col)

                # Now we can store the SharedString value
                cell_value = cell_ref.value
                row_contents.append(cell_value)
            cell_contents.append(row_contents)
        
        # Write cell contents to the docx using add_to_paragraph()
#         cell_value.add_to_paragraph(paragraph)
        
        return cell_contents
    
    
    # Openpyxl
    # Note: openpyxl cannot read/copy charts; it needs to recreate them from source data
    # Read-only and data-only increases the speed of reading data from workbooks
    wb = load_workbook(source, data_only=True)
    
    table_values = cell_text_runs()
    
    # Loop through each worksheet
    for ws in wb.worksheets:

        # Get dimensions of table in Excel
        data_vals = np.asarray(tuple(ws.values))  # Will need to add logic to read only the table and not comments
        table_dim = np.shape(data_vals)

        # Get merged ranges
        merged_ranges = ws.merged_cells.ranges

        # Store dict of formats
        src_fmts = {}
        for r,row in enumerate(ws.rows):
            for c,cell in enumerate(row):
                src_fmts[(r,c)] = {
                    'bold': cell.font.b,
                    'italic': cell.font.i,
                    'name': cell.font.name,
                    'size': cell.font.size,
                    'fillColor': xl2doc_color(cell.fill.start_color),
                    'fontColor': xl2doc_color(cell.font.color),
                    'horizontal': cell.alignment.horizontal,
                    'vertical': cell.alignment.vertical,  # can build overrides
                    'border': {
                        'top': cell.border.top.style,
                        'topColor': xl2doc_color(cell.border.top.color),
                        'bottom': cell.border.bottom.style,
                        'bottomColor': xl2doc_color(cell.border.bottom.color),
                        'left': cell.border.left.style,
                        'leftColor': xl2doc_color(cell.border.left.color),
                        'right': cell.border.right.style,
                        'rightColor': xl2doc_color(cell.border.right.color),
                    }
                }
        # Docx
        new_section_cols(1)  # Ensure Word section has only one column

        # Add heading if required
        if heading:
            dest.add_paragraph(style='Heading 1').add_run().add_text(heading)

        # Create, table in word
        table = dest.add_table(rows=table_dim[0], cols=table_dim[1])
        
        # Merge table cells if any found in Excel
        if len(merged_ranges):
            for merged_range in merged_ranges:
                start_cell = table.cell(
                    merged_range.min_row-1,
                    merged_range.min_col-1
                )
                end_cell = table.cell(
                    merged_range.max_row-1,
                    merged_range.max_col-1
                )
                start_cell.merge(end_cell)
                
        # Write to table
        for r, row in enumerate(table.rows):
            for c, cell in enumerate(row.cells):
                if len(table_values[r][c].plain_text()) > 0:
                    table_values[r][c].add_to_paragraph(cell.paragraphs[0])
                    
        # Style table
        style_tbl(table, src_fmts)


print('append_xlsx() code')
hide_toggle()

append_xlsx() code


In [10]:
def append_docx(dest, data, columns=1, new_page=False, separate_header=False):
    '''Appends content from the Word source to the destination Word doc - supports text and in-line images.
    DOES NOT SUPPORT FLOATING IMAGES AND SHAPES! Use add_docx() instead
    
    Parameters
    ----------
    dest : str
        The file location of the destination word doc
    source: str
        The file location of the target Word source
    '''
    source = docx.Document(data)
    ims = [im for im in source.inline_shapes]

    # Persistent indexes
    im_paths = []
    im_heights = []
    im_widths = []

    # Temp variables
    im_streams = []
    
    # New page if true
    if new_page:
        dest.add_paragraph().add_run().add_break(WD_BREAK.PAGE)
    
    # Get image binary and metadata
    for im_idx, im in enumerate(ims):
        # Binary
        blip = im._inline.graphic.graphicData.pic.blipFill.blip
        rId = blip.embed
        doc_part = source.part
        image_part = doc_part.related_parts[rId]
        byte_data = image_part._blob
        image_stream = io.BytesIO(byte_data)
        im_streams.append(image_stream)

        # Metadata
        image_name = f'img_{im_idx}.jpeg'
        im_heights.append(im.height.mm)
        im_widths.append(im.width.mm)
        
        # Save images to temp folder
        im_path = os.path.join(temp_path,image_name)
        im_paths.append(im_path)
        with open(im_path, "wb") as fh:
            fh.write(byte_data)
        fh.close()
    # ----End image transfer----
    # ----Start copying docx content----
    # Populate and save output
    paras = source.paragraphs
    im_idx = 0
    
    # Split into columns if header is not separate
    # Otherwise, split into columns after the header
    if not separate_header:
        new_section_cols(dest, columns)
    
    for para_idx, para in enumerate(paras):
        if(para.text):
            get_para_data(dest, para)
        
        # Copy images over
        root = ET.fromstring(para._p.xml)
        namespace = {'wp':"http://schemas.openxmlformats.org/drawingml/2006/wordprocessingDrawing"}
        inlines = root.findall('.//wp:inline',namespace)

        if(len(inlines) > 0):
            uid = f'img_{im_idx}'

            img = dest.add_paragraph()
            img.add_run().add_text("{{ " + uid + " }}")
            img.alignment = WD_ALIGN_PARAGRAPH.CENTER

            context[uid] = InlineImage(
                dest,
                im_paths[im_idx],
                width=Mm(im_widths[im_idx]),
                height=Mm(im_heights[im_idx]),
            )
            im_idx += 1
            
            
        # Split into columns after the header
        if (para_idx == 0 and separate_header):
            new_section_cols(dest, columns)
            
print('append_docx() code')
hide_toggle()

append_docx() code


## Main

In [40]:
# SETUP
# Get paths to sample content and data
sample_path = os.path.join(os.curdir,'samples')
contents = []  # Initialise an empty list to store paths to contents files

for dirname, dirnames, filenames in os.walk(sample_path):
    for fname in filenames:
        if 'content' in fname:
            contents.append(os.path.join(dirname,fname))
        elif 'backpage' in fname:
            backpage = os.path.join(dirname,fname)

# Sort contents list
contents.sort()
            
# Create empty output file as placeholder
output = docx.Document()
output_path = os.path.join(sample_path,'0 output.docx')
output.save(output_path)

# Create temp folder. Folder deleted after publishing
# Use tempfile when code is rebased
temp_path = os.path.join(os.curdir,'temp')
try:
    os.mkdir(temp_path)
except FileExistsError as e:
    print(f'{e}: Folder already exists...continuing\n')
    pass

# Initiate template path to title page and content to fill title+backpage
title_page = os.path.join(sample_path,'1 template.docx')
context = {
    'title': 'Prototyping with Bob',
    'subtitle': 'Prepared by Yemeng Bob Jin for Yeqin Jim Jin',
    'date': datetime.date.today(),
    'closing': 'THANK YOU',
    'copyright': 'Give me a shout out and you can do whatever (GNU Licence)',
    'website': 'www.bobjin.me',
    'email': 'automaticjinandtonic@gmail.com',
    'number': '+61 4XX XXX XXX'
}


# __MAIN__
dest = DocxTemplate(title_page)  # Setup template
dest.add_paragraph().add_run().add_break(WD_BREAK.PAGE)  # Go to a new page

# Append all content to destination
for content in contents:
    print(content)
    if 'xlsx' in content:
        if 'tbl' in content:
            append_xlsx(dest, content)
        elif 'chart' in content:
            pass  # placeholder for charting function
        # Add space after Excel table
        dest.add_paragraph().paragraph_format.space_after = Pt(20)
    elif 'docx' in content:
        cols = content[:-5].split('_')[-1]  # Read from filename...need a better way of doing this
        append_docx(dest, content, columns=cols, separate_header=True)
publish()

.\samples\sample_content_0_cols_1.docx
.\samples\sample_content_1_tbl.xlsx
.\samples\sample_content_2_cols_2.docx
.\samples\sample_content_3_tbl.xlsx
Saved at .\samples\0 output.docx


### Issues List

- Start packaging this project into proper modules
    - Get the definition of modules and packages downpat
    - Also, consider learning lambda functions?
- Check_styles() - Formatting parser during publishing...needs a custom solution?
    - Ensure:
        - There is not too much blank space on a page
        - Tables and titles are not left orphaned on a page
    - Consistent formatting throughout
- Chart_xlsx() - Excel chart creation | [Documentation](https://openpyxl.readthedocs.io/en/stable/charts/introduction.html)
    - [Reader drawings module](https://openpyxl.readthedocs.io/en/stable/api/openpyxl.reader.drawings.html)


### Longer-term issues
- Front-end | Parameter entry:
    - Manual entry
    - Auto-load in
- Front-end <--link--> Back-end
    - Where will files be stored? Privacy issues?
- Flexibly defined style formats?
- Creation of some standard templates
    - Equity f/s
    - Fixed income f/s
    - Fund f/s
- Scheduler (.bat?)
- Delivery method
    - Email, file saved
    - Cloud-based vs Local?